In [ ]:
pip install -r ../requirements.txt

In [1]:
import pandas as pd
from tabula import read_pdf
import numpy as np
import os

In [ ]:
# Extract the PDF file into a list of DataFrames
input_path = "../data/men/raw/CED-11-04-23.pdf"
output_path = "../data/men/clean/23-24/CED-11-04-23/"

# Label Team Names
team_name = 'Mo.-St. Louis'
opp_team_name = 'Cedarville'

# Choose ranges of data to scrape from PDF
team_range = range(20, 29)
opp_team_range = range(0, 10)

dfs = read_pdf(input_path, pages="all", multiple_tables=True)

# Separate the DataFrames into individual variables
df0 = dfs[0]
df1 = dfs[1]

In [ ]:
# Find range of rows for each team (ranges begin on [0:?] and [20:?])
#df1[0:10]
#df1[20:29]

In [5]:
def process_score(df):
    # Drop the first column and the second row, rename columns
    df = df.iloc[:, 1:].iloc[[0, 2], :].rename(
        columns={'Unnamed: 0': 'Team', 
                 'Unnamed: 1': '1st Half',
                 'Unnamed: 2': '2nd Half',
                 'Unnamed: 3': 'Final'})
    
    # Convert values to integers
    df['1st Half'] = df['1st Half'].astype(int)
    df['2nd Half'] = df['2nd Half'].astype(int)
    df['Final'] = df['Final'].astype(int)
    
    return df



# Clean the Player Stats tables for both teams
def process_stats(df):
    # Create an explicit copy of the DataFrame
    df = df.copy()
    
    # Remove rows with all NaN values
    df = df.dropna(axis=1, how='all')

    # Split 'FT ORB-DRB' column into separate columns
    df[['FT', 'ORB-DRB']] = df['FT ORB-DRB'].str.split(' ', expand=True)

    # Split 'ORB-DRB' column into separate columns
    df[['ORB', 'DRB']] = df['ORB-DRB'].str.split('-', expand=True).astype(float)

    # Split FG, 3PT, and FT columns into made and attempted
    df[['FGM', 'FGA']] = df['FG'].str.split('-', expand=True).astype(float)
    df[['3PTM', '3PTA']] = df['3PT'].str.split('-', expand=True).astype(float)
    df[['FTM', 'FTA']] = df['FT'].str.split('-', expand=True).astype(float)

    # Split 'A TO BLK' column into 3 separate columns
    df[['A', 'TO', 'BLK']] = df['A TO BLK'].str.split(' ', expand=True).astype(float)

    # Split 'GS MIN' column into 2 separate columns
    df['MIN'] = df['GS MIN'].str.replace('*', '').str.replace(' ', '').astype(float)

    # Split '# Player' column into separate columns
    df[['#', 'Player']] = df['# Player'].str.split(' ', n=1, expand=True)
    # Clean name values
    df['Player'] = df['Player'].str.split(',').str[::-1].str.join(' ')

    # Convert rest of relevant columns to float type
    df['PTS'] = df['PTS'].astype(float)
    df['REB'] = df['REB'].astype(float)
    df['STL'] = df['STL'].astype(float)
    df['PF'] = df['PF'].astype(float)
    
    # Calculate Advanced Statistics
    # eFG% (Effective Field Goal Percentage): Measures shooting efficiency, taking into account 3-pointers. Formula: (FGM + 0.5*3PM) / FGA
    df['eFG%'] = (df['FGM'] + (0.5 * df['3PTM'])) / df['FGA']
    df['eFG%'] = round(df['eFG%'], 2)

    # TS% (True Shooting Percentage): Measures shooting efficiency, taking into account 3-pointers and free throws. Formula: PTS / (2*(FGA + 0.44*FTA))
    df['TS%'] = df['PTS'] / (2 * (df['FGA'] + (0.44 * df['FTA'])))
    df['TS%'] = round(df['TS%'], 2)

    # Assist-to-Turnover Ratio: Measures the number of assists per turnover
    df['A/TO'] = np.where(df['TO'] == 0, 0, df['A'] / df['TO']) # Numpy handles div by 0 cases were 0 Turnovers(TO) are committed
    df['A/TO'] = round(df['A/TO'], 2)

    # Usage Rate: Measures how often a player is involved in team plays
    df['Usage Rate'] = (df['FGA'] + df['FTA'] + df['A'] + df['TO']) / (df['MIN'] / 40)  # 40 minutes per game
    df['Usage Rate'] = round(df['Usage Rate'], 2)

    # Box Plus/Minus: Measures a player's overall contribution
    df['Box +/-'] = np.where(df['MIN'] == 0, 0, ((df['PTS'] + df['REB'] + df['A'] + df['STL'] + df['BLK']) - (df['FGA'] + df['FTA'] + df['TO'] + df['PF'])) / df['MIN']) # Numpy handles div by 0 cases were 0 Minutes(MIN) are recorded
    df['Box +/-'] = round(df['Box +/-'], 2)

    # Drop original columns that were cleaned
    df = df.drop(['FT ORB-DRB', 'A TO BLK', 'ORB-DRB', 'FG', '3PT', 'FT', 'GS MIN', '# Player', '#'], axis=1)
    # Place columns in a specific order
    df = df[
        ['Player', 'MIN', 'PTS', 'FGM', 'FGA', '3PTM', '3PTA', 'FTM', 
         'FTA', 'A', 'REB', 'ORB', 'DRB', 'TO', 'BLK', 'STL', 'PF', 'eFG%', 'TS%', 'A/TO', 'Usage Rate', 'Box +/-']]

    return df


score = process_score(df0)
playerStats = process_stats(df1.iloc[team_range])
oppPlayerStats = process_stats(df1.iloc[opp_team_range])

playerStats = playerStats.assign(Team=team_name, Opponent=opp_team_name)
oppPlayerStats = oppPlayerStats.assign(Team=opp_team_name, Opponent=team_name)


In [6]:
score

,Team,1st Half,2nd Half,Final
0,Cedarville,40,35,75
2,Mo.-St. Louis,27,43,70


In [7]:
playerStats

,Player,MIN,PTS,FGM,FGA,3PTM,3PTA,FTM,FTA,A,...,BLK,STL,PF,eFG%,TS%,A/TO,Usage Rate,Box +/-,Team,Opponent
20,Matt Enright,36.0,22.0,9.0,15.0,0.0,5.0,4.0,5.0,2.0,...,0.0,2.0,4.0,0.60,0.64,2.00,25.56,0.14,Mo.-St. Louis,Cedarville
21,Janeir Harris,38.0,14.0,5.0,11.0,0.0,3.0,4.0,5.0,3.0,...,2.0,1.0,2.0,0.45,0.53,3.00,21.05,0.13,Mo.-St. Louis,Cedarville
22,Mayson Quartlebaum,24.0,13.0,6.0,10.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.65,0.65,0.00,18.33,0.29,Mo.-St. Louis,Cedarville
23,Emanuel Prospere II,37.0,9.0,4.0,7.0,1.0,2.0,0.0,2.0,4.0,...,0.0,2.0,2.0,0.64,0.57,2.00,16.22,0.16,Mo.-St. Louis,Cedarville
24,Terrell Kabala,23.0,6.0,2.0,6.0,2.0,4.0,0.0,0.0,2.0,...,0.0,1.0,3.0,0.50,0.50,2.00,15.65,0.04,Mo.-St. Louis,Cedarville
25,Troy Glover II,16.0,6.0,2.0,3.0,0.0,0.0,2.0,4.0,0.0,...,1.0,1.0,2.0,0.67,0.63,0.00,20.00,0.25,Mo.-St. Louis,Cedarville
26,Kris O'Neal II,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,3.0,1.0,0.00,0.00,0.33,18.18,0.09,Mo.-St. Louis,Cedarville
27,Christian Meeks,8.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,15.00,-0.12,Mo.-St. Louis,Cedarville
28,Sam Bledsoe,7.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,17.14,-0.43,Mo.-St. Louis,Cedarville


In [8]:
oppPlayerStats

,Player,MIN,PTS,FGM,FGA,3PTM,3PTA,FTM,FTA,A,...,BLK,STL,PF,eFG%,TS%,A/TO,Usage Rate,Box +/-,Team,Opponent
0,Jayvon Maughmer,33.0,31.0,11.0,18.0,3.0,5.0,6.0,6.0,0.0,...,1.0,0.0,2.0,0.69,0.75,0.00,31.52,0.33,Cedarville,Mo.-St. Louis
1,Chris Rogers,30.0,14.0,5.0,8.0,2.0,5.0,2.0,2.0,3.0,...,0.0,0.0,2.0,0.75,0.79,3.00,18.67,0.13,Cedarville,Mo.-St. Louis
2,Grant Whisman,31.0,11.0,4.0,9.0,3.0,7.0,0.0,0.0,1.0,...,0.0,0.0,3.0,0.61,0.61,0.00,12.90,0.06,Cedarville,Mo.-St. Louis
3,Jacob Drees,24.0,4.0,2.0,5.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.40,0.40,0.25,16.67,0.21,Cedarville,Mo.-St. Louis
4,Timothy Davis,6.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.00,0.00,0.00,26.67,-1.17,Cedarville,Mo.-St. Louis
5,Kyle Thomas,29.0,9.0,3.0,7.0,3.0,7.0,0.0,0.0,2.0,...,0.0,1.0,0.0,0.64,0.64,2.00,13.79,0.28,Cedarville,Mo.-St. Louis
6,Anthony Ruffolo,16.0,4.0,2.0,4.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.50,0.50,1.00,15.00,0.12,Cedarville,Mo.-St. Louis
7,Tymoteusz Pszczola,14.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.25,0.25,0.00,14.29,-0.07,Cedarville,Mo.-St. Louis
8,Ethan Sellars,16.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,...,0.0,0.0,1.0,0.00,0.00,1.00,15.00,-0.06,Cedarville,Mo.-St. Louis
9,David Okpara,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,NaN,NaN,0.00,0.00,-1.00,Cedarville,Mo.-St. Louis


In [9]:
# Export function
def export_to_csv(output_path, score, playerStats, oppPlayerStats):
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Export DataFrames to CSV files
    score.to_csv(os.path.join(output_path, "score.csv"), index=False)
    playerStats.to_csv(os.path.join(output_path, "playerStats.csv"), index=False)
    oppPlayerStats.to_csv(os.path.join(output_path, "oppPlayerStats.csv"), index=False)

# Call the function with your DataFrames and output path
export_to_csv(output_path, score, playerStats, oppPlayerStats)